In [29]:
import networkx as nx

from fair_cc_functions import *
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [3]:
def greedy_puzzle(red_g, blue_g, iterations=10):
    # 2) perform cc-pivot on each subgraph, clusters sorted
    blue_cluster = sorted(cc_pivot(blue_g), key=len)
    red_cluster = sorted(cc_pivot(red_g), key=len)

    # 3) get size of clustering
    min_c = np.min([len(blue_cluster), len(red_cluster)])
    max_c = np.max([len(blue_cluster), len(red_cluster)])
    if len(red_cluster) > len(blue_cluster):
        big_c = red_cluster
        small_c = blue_cluster
    else:
        big_c = blue_cluster
        small_c = red_cluster

    # 4) greedy algo and sort the largest clusters together
    final_cluster=[]
    for i in range(1, min_c+1):
        final_cluster.append(big_c[-i] + small_c[-i])

    for i in range(max_c-min_c-1, -1, -1):
        final_cluster[-((i)%min_c+1)].extend(big_c[i])
    return final_cluster, red_cluster, blue_cluster

In [34]:
def stats_puzzle(n, red_type, blue_type, iters=10):
    # 1) generate 2 subgraphs (one red, one blue)
    cluster_bal = []
    red_cost, blue_cost = [], []
    cluster_cost = []
    for i in range(10, n, 2):
        _, red_g, blue_g = generate_red_blue_graph(i, red_type, blue_type)
        red_g.add_edges_from(red_g.edges, weight=1)
        red_g.add_edges_from(nx.complement(red_g).edges, weight=0)
        blue_g.add_edges_from(blue_g.edges, weight=1)
        blue_g.add_edges_from(nx.complement(blue_g).edges, weight=0)
        composed_graph = nx.compose(red_g, blue_g)
        bal, c, cost_b, cost_r = [], [], [], []
        for j in range(iters):
            fin_c, r_cl, b_cl = greedy_puzzle(red_g, blue_g)
            bal.append(balance(fin_c, composed_graph))
            c.append(cost(fin_c, composed_graph))
            cost_b.append(cost(b_cl, blue_g))
            cost_r.append(cost(r_cl, red_g))
        cluster_bal.append(bal)
        cluster_cost.append(c)
        red_cost.append(cost_r)
        blue_cost.append(cost_b)
    return cluster_bal, cluster_cost, red_cost, blue_cost

In [35]:
star_star_bal, star_star_c, red_star_c, blue_star_c = stats_puzzle(61, 'star', 'star', iters=20)

In [61]:
# probs = []
# m = 4
# for m in range(1, 100):
#     prob = 0
#     for i in range(1, m+1):
#         prob_m=1
#         for j in range(m, m-i, -1):
#             prob_m = prob_m*(1/(j*j))
#         prob += prob_m
#     probs.append(prob)
# x = [1/i for i in range(1, 100)]